## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-06-30-36 +0000,nypost,"Missing Long Island boy, 15, may have traveled...",https://nypost.com/2026/01/24/us-news/missing-...
1,2026-01-24-06-22-08 +0000,nypost,Texas posthumously exonerates Tommy Lee Walker...,https://nypost.com/2026/01/24/us-news/texas-ma...
2,2026-01-24-06-03-18 +0000,nypost,Florida killer thrown out of courtroom over ou...,https://nypost.com/2026/01/24/us-news/florida-...
3,2026-01-24-05-52-03 +0000,nypost,HUD orders public housing officials to verify ...,https://nypost.com/2026/01/24/us-news/hud-orde...
4,2026-01-24-05-36-01 +0000,nypost,"Thousands march through Minneapolis, swarm Tar...",https://nypost.com/2026/01/24/us-news/thousand...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2446/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
148,trump,57
55,ice,22
104,greenland,17
105,new,11
76,china,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
280,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...,101
37,2026-01-24-02-00-00 +0000,wsj,Trump’s aggressive deployment of ICE has thrus...,https://www.wsj.com/politics/policy/democrat-r...,98
236,2026-01-23-15-04-40 +0000,wapo,"On Greenland, Europe stood up, Trump blinked, ...",https://www.washingtonpost.com/world/2026/01/2...,95
201,2026-01-23-17-00-09 +0000,latimes,Trump lawyers urge Supreme Court to block Cali...,https://www.latimes.com/politics/story/2026-01...,94
290,2026-01-23-10-00-24 +0000,wapo,Trump says China is a threat to Greenland. Wha...,https://www.washingtonpost.com/world/2026/01/2...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
280,101,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...
5,59,2026-01-24-05-31-24 +0000,startribune,"The latest: Thousands march, scores of Minneso...",https://www.startribune.com/ice-raids-minnesot...
302,43,2026-01-23-08-08-54 +0000,bbc,Is the US-TikTok deal a new reality for China'...,https://www.bbc.com/news/articles/c6200mz054yo...
74,41,2026-01-23-23-34-51 +0000,nypost,Ex-NJ Gov. Phil Murphy granted clemency to mom...,https://nypost.com/2026/01/23/us-news/murphy-g...
125,38,2026-01-23-21-39-00 +0000,wsj,Intel Shares Slide as Costs Pile Up in Bid to ...,https://www.wsj.com/business/earnings/intel-in...
47,36,2026-01-24-01-11-10 +0000,nyt,"Ryan Wedding, Canadian Ex-Snowboarder Accused ...",https://www.nytimes.com/2026/01/23/world/canad...
187,33,2026-01-23-18-10-10 +0000,nypost,DHS officials debunk viral claims ICE used 5-y...,https://nypost.com/2026/01/23/us-news/dhs-offi...
226,31,2026-01-23-15-42-40 +0000,wapo,Trump dominated Davos. But Canada’s Carney was...,https://www.washingtonpost.com/world/2026/01/2...
87,28,2026-01-23-23-02-31 +0000,nyt,FBI Agent Resigns After Trying to Investigate ...,https://www.nytimes.com/2026/01/23/us/fbi-agen...
238,28,2026-01-23-14-59-43 +0000,bbc,Trump 'wrong' to claim Nato troops avoided Afg...,https://www.bbc.com/news/articles/czr444j671vo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
